CUDA(Compute Unified Device Architecture)，是NVIDIA公司推出的通用并行计算架构，使得我们可以方便的使用GPU解决复杂的计算问题。Pytorch中通过**torch.cuda**来管理和运行CUDA操作。一个tensor被分配到某块显卡上后，整个运算周期都在该显卡上进行，除非使用了显示的copy_()操作，否则不允许跨显卡的tensor操作。另外，具有专门的语句实现数据在主板内存与显卡显存之间的交换。

In [ ]:
import torch

In [ ]:
cuda = torch.device('cuda')    # 使用默认的CUDA设备
cuda0 = torch.device('cuda:0')
cuda2 = torch.device('cuda:0')# GPU 2（从0开始计数）

x = torch.tensor([1,2],device = cuda0) # 在GPU 0 上分配一个tensor
y = torch.tensor([1,2]).cuda() # 将tensor　从CPU转移到GPU,　.cuda()中也可指定具体设备，例如.cuda(cuda0)
z = torch.tensor([1,2]).to(device=cuda0) #　使用Tensor.to()语法来将数据从CPU转移到GPU

with torch.cuda.device(1):
    ## 可先指定明确的设备号，然后是用默认的cuda即可
    a = torch.tensor([1,2],device = cuda)
    b = torch.tensor([1,2]).cuda()
    c = torch.tensor([1,2]).to(device = cuda)
    
    # 也可在该with的缩进范围内另外指定具体的设备，这种指定具有更高的优先级
    d = torch.tensor([1,2],device = cuda2)
    e = torch.tensor([1,2]).cuda(cuda2)
    f = torch.randn([1,2]).to(device = cuda2)

默认情况下，GPU的操作都是异步的，当我们调用一个在GPU上执行的函数时，其会添加进GPU的队列中等待执行，此时并不影响CPU和其他GPU上的操作。一般来说，GPU异步执行的效果对我们来说是不可见的，直观上我们会认为其是在同步执行的，其原因有二：

-　每一个GPU设备都会顺序执行队列里的操作
-　Pytorch会在CPU与GPU之间，GPU与GPU之间拷贝数据的时候进行必要的同步

我们可以通过设置环境变量**CUDA_LAUNCH_BLOCKING=1**来强制同步操作，这对于代码的调试是很有用的，例如，在异步执行情况下，GPU上的错误直到这个操作完全执行完后才会报出，这时的栈就很难准确指出出错的位置。

一些操作例如**to()** 和 **copy_()** 允许明确的 **non_blocking**参数，这会让调用者忽略到不必要的同步。另一个是CUDA流

#### CUDA Streams

CUDA流是属于某一个具体设备的执行序列，每个设备都有一个默认的CUDA流，另外也可创建新的流。每个CUDA流中的操作都是按照创建的顺序执行的，但是不同流之间的操作可能以各种相对顺序操作。如果当前的流是默认流，PyTorch会在数据传输时自动完成必要的同步操作，但是，如果使用了非默认流，则需要用户自己明确指定同步操作。下述是一个错误操作的例子：

In [ ]:
cuda = torch.device('cuda')
s = torch.cuda.Stream()   #新创建的流
A = torch.empty((100,100),device = cuda).normal_(0.0,1.0)
with torch.cuda.stream(s):
    B = torch.sum(A)

PyTorch使用缓存分配器来加速内存分配，这会在无需设备同步的情况下快速释放内存。然而，当我们使用**nvidia-smi** 进行监控的时候，有分配器管理的未使用的内存仍会被显示在使用中。

- 使用**memory_allocated()**　和　**max_memeory_allocated()**　来监控被tensor占用的内存
- 使用**memory_cached()** 和　**max_memory_cached()**　来监控被缓存分配器管理的内存。

使用**empty_cache()**　可以释放未使用的缓存，这些缓存可以被其他的GPU应用使用，但是不能释放被tensor占用的GPU内存。

#### Best Practices

In [ ]:
import argparse
import torch

parser = argparse.ArgumentParser(description='Pytorch Example')
parser.add_argument('--disable_cuda', action = 'store_true',help='Disable CUDA')
args = parser.parse_args()

args.device = torch.device("cuda" if not args.disable_cuda and torch.cuda.is_available() else "cpu")

x = torch.empty((8,42),device = args.device)
net = Network().to(device=args.device)

for i, x in enumerate(train_loader):
    x = x.to(args.device)

如果已经有了一个tensor,且想要在相同的设备上创建一个相同类型的tensor，可以使用**torch.Tensor.new_*** 方法。如果想要新建一个相同大小和类型的tensor,且用０或者１来填充，可以使用**torch.zeros_like()** 和 **torch.ones_like()**　方法。

In [ ]:
cuda = torch.device('cuda')
x_cpu = torch.empty(2)
x_gpu = torch.empty(2,device = cuda)
x_cpu_long = torch.empty(2,dtype = troch.int64)

y_cpu = x_cpu.new_full([3,2],fill_value = 0.3)
y_gpu = x_gpu.new_full([3,2],full_value = -5)
y_cpu_long = x_cpu_long.new_tensor([[1,2,3]])

In [ ]:
x_cpu = torch.empty(2,3)
x_gpu = torch.empty(2,3)

y_cpu = torch.ones_like(x_cpu)
y_gpu = torch.zeros_like(x_gpu)

#### 锁页内存（pin memory）

主机中的内存分为两种：锁页和非锁页。锁页内存存放的内容无论什么时候都不会与虚拟内存交换，而非锁页内存在主机内存不足是，数据会放置在锁页内存中。显卡中的内存都是锁页内存。使用Dataloader时，如果设置**pin_memory=True**,则生成的tensor一开始在主机内存中就是锁页的，因此在将其转移到GPU显存是的速度就会更快一些。一般来说我们的台式机内存资源都够用，因此可以设置**pin_memory=True**

当有多个GPU资源时，使用nn.DataParallel，而不要使用multiprocessing